In [36]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq
from datasets import load_dataset
import cassio
from PyPDF2 import PdfReader

astra_db_token = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
astra_id = os.getenv("ASTRA_ID")
groq_api = os.getenv("GROQ_API")
hf_token = os.getenv("HF_TOKEN")

In [3]:
pdfreader = PdfReader(r'C:\Users\Raghu\Downloads\Gen AI Projects\pdfbot\basic-laws-book-2016.pdf')

from typing_extensions import Concatenate

raw_text = ''
for i,page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [26]:
cassio.init(token="AstraCS:qpHtvznmeuyqrZrqOXXbiaSn:8a758fd88cf7c4c9e332ddc9ae51e97449de4fba23d43fdc8b795eeca07781e3",database_id="b3c61e69-216f-4f17-a126-f56fc9793b6b")

In [31]:
llm = ChatGroq(model="gemma2-9b-it",api_key=os.getenv("GROQ_API"))
embeddings = OllamaEmbeddings(model="llama3.1")

C:\Users\Raghu\AppData\Local\Temp\ipykernel_20184\3048509702.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3.1")


In [32]:
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="qa_mini_proj",
    session=None,
    keyspace=None
)

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(separators="\n\n",chunk_size = 900,chunk_overlap = 200,length_function = len)

text_chunks = text_splitter.split_text(raw_text)
type(text_chunks[0])

str

In [37]:
astra_vector_store.add_texts(text_chunks[:50])
print(f"Inserted %i headlines." % len(text_chunks[:50]))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


In [47]:
Markdown(text_chunks[199])

(a) Provisions pertaining to the Review Board.—The pro -
visions of this Act that pertain to the appointment and 
operation of the Review Board shall cease to be effective 
when the Review Board and the terms of its members 
have terminated pursuant to section 7(o).
(b) Other provisions.—The remaining provisions of this 
Act shall continue in effect until such time as the Archivist 
certifies to the President and the Congress that all assassi -
nation records have been made available to the public in 
accordance with this Act.
Sec. 13. Authorization of appropriations
(a) In general.—There are authorized to be appropriated 
to carry out the provisions of this Act $1,600,000 for fis -
cal year 1998.
(b) Interim funding.—Until such time as funds are ap -
propriated pursuant to subsection (a), the President may 
use such sums as are available for discretionary use to car -
ry out this Act.

In [54]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Define the prompt template
query_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
Based on the document content, provide a detailed response to the following question.

Query: {query}

Ensure the answer is based strictly on the available document text. If the requested information is not in the document, respond with "This document does not contain the answer to this question."

Response:
""",
)
# Initialize the parser
output_parser = StrOutputParser()

# Example usage
query_text = "What is the authorization for appropriations in this Act?"
chain = query_prompt | llm | output_parser
response = chain.invoke({"query":query_text})
Markdown(response)

Please provide the document so I can answer your question. 
